# Set-up

Change the working directory to the one where you saved your files

In [40]:
cd /content/drive/MyDrive/COMP0087/allenNLP/BCN

/content/drive/MyDrive/COMP0087/allenNLP/BCN


In [38]:
test_set = pd.read_json('/content/drive/MyDrive/COMP0087/data/test.jsonl', orient='records', lines=True)

In [15]:
%%shell
pip install torch==1.7.1

In [39]:
from torch import nn
import torch.optim as optim
import numpy as np
import spacy
nlp = spacy.load('en_core_web_sm')
import pandas as pd

In [8]:
%%shell
pip install allennlp==2.1.0 allennlp-models==2.1.0

     |████████████████████████████████| 593kB 5.6MB/s 
     |████████████████████████████████| 409kB 34.2MB/s 
     |████████████████████████████████| 102kB 9.9MB/s 
     |████████████████████████████████| 266kB 33.6MB/s 
     |████████████████████████████████| 1.2MB 33.0MB/s 
     |████████████████████████████████| 12.8MB 219kB/s 
     |████████████████████████████████| 1.9MB 16.6MB/s 
     |████████████████████████████████| 122kB 33.0MB/s 
     |████████████████████████████████| 776.8MB 24kB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 7.4MB 36.9MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 3.3MB 34.7MB/s 
     |████████████████████████████████| 870kB 33.4MB/s 
  Created wheel for boto3: filename=boto3-1.17.46-py2.py3-none-any.whl size=128779 sha256=cdd6ac8d58e97e326cc43295763a83ec683795fa2482b16feb13a71f6cbcac91
  Stored in d

In [3]:
# imports from allennlp
from allennlp.models.archival import load_archive
from allennlp.common.util import JsonDict
from allennlp.data import Instance
from allennlp.predictors.predictor import Predictor
from allennlp.data.fields import LabelField
from allennlp.data.tokenizers.spacy_tokenizer import SpacyTokenizer

from typing import List, Dict

from overrides import overrides

from allennlp.interpret.attackers import Attacker, InputReduction

from allennlp.interpret.saliency_interpreters import SimpleGradient

In [5]:
%%shell
pip install checklist==0.0.10

     |████████████████████████████████| 12.1MB 250kB/s 
     |████████████████████████████████| 22.3MB 2.1MB/s 
     |████████████████████████████████| 174kB 42.0MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 5.6MB 26.0MB/s 
     |████████████████████████████████| 5.5MB 40.9MB/s 
     |████████████████████████████████| 419kB 37.1MB/s 
     |████████████████████████████████| 3.2MB 39.6MB/s 
     |████████████████████████████████| 102kB 10.5MB/s 
  Created wheel for checklist: filename=checklist-0.0.10-cp37-none-any.whl size=12164434 sha256=69d32ecf0d53ce66c04afb59bd2da784568fe33a73b95786b766133aeaf1dd15
  Stored in directory: /root/.cache/pip/wheels/c6/a2/02/f075a28620a8370dd79df36bb486095d465e7870623cea6b4e
  Created wheel for patternfork-nosql: filename=patternfork_nosql-3.6-cp37-none-any.whl size=22332807 sha256=1ed0550cfa61dfd097da0fd499ee6f4aa61dc640948583a20202e140e053393e
  Stored in directory: /root/.cache/pip/wheels/dc/0d/ae/

In [6]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
from scipy.stats import kendalltau
from scipy.stats import spearmanr

Background about model: http://docs.allennlp.org/v0.9.0/api/allennlp.models.biattentive_classification_network.html

In [15]:
# importing the dataset reader
import tagging
# importing the BCN model
import BCN_model

# Training

In [16]:
# training model
# here, the output will be saved to a new folder called 'BCN_output'. You will get an error message if such a directory already exists.
!pwd; allennlp train --include-package tagging -s BCN_output config_BCN.jsonnet

/content/drive/MyDrive/COMP0087/allenNLP/BCN
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2021-04-07 10:16:15,656 - ERROR - allennlp.common.plugins - Plugin allennlp_models could not be loaded: No module named 'nltk.translate.meteor_score'
2021-04-07 10:16:16,443 - INFO - allennlp.common.params - include_in_archive = None
2021-04-07 10:16:16,449 - INFO - allennlp.common.params - random_seed = 13370
2021-04-07 10:16:16,450 - INFO - allennlp.common.params - numpy_seed = 1337
2021-04-07 10:16:16,450 - INFO - allennlp.common.params - pytorch_seed = 133
2021-04-07 10:16:16,519 - INFO - allennlp.common.checks - Pytorch version: 1.7.1
2021-04-07 10:16:16,520 - INFO - allennlp.common.params - type = default
2021-04-07 10:16:16,521 - INFO - allennlp.common.params - dataset_reader.type = ag_reader
2021-04-07 10:16:16,522 -

# Predictions

In [17]:
@Predictor.register('ag_text_classifier')
class AGNewsClassifier(Predictor):
    """
    Predictor for any model that takes in a sentence and returns
    a single class for it.  In particular, it can be used with
    the [`BasicClassifier`](../models/basic_classifier.md) model.

    """

    def predict(self, sentence: str) -> JsonDict:
        return self.predict_json({"Description": sentence})

    @overrides
    def _json_to_instance(self, json_dict: JsonDict) -> Instance:
        """
        Expects JSON that looks like `{"sentence": "..."}`.
        Runs the underlying model, and adds the `"label"` to the output.
        """
        sentence = json_dict["Description"]
        reader_has_tokenizer = (
            getattr(self._dataset_reader, "tokenizer", None) is not None
            or getattr(self._dataset_reader, "_tokenizer", None) is not None
        )
        if not reader_has_tokenizer:
            tokenizer = SpacyTokenizer()
            sentence = tokenizer.tokenize(sentence)
        return self._dataset_reader.text_to_instance(sentence)

    @overrides
    def predictions_to_labeled_instances(
        self, instance: Instance, outputs: Dict[str, np.ndarray]
    ) -> List[Instance]:
        new_instance = instance.duplicate()
        label = np.argmax(outputs["class_probabilities"])
        new_instance.add_field("label", LabelField(int(label), skip_indexing=True))
        return [new_instance]

In [18]:
archive = load_archive("./BCN_output/model.tar.gz")
model = archive.model
vocab = model.vocab

In [41]:
predictor = Predictor.from_archive(archive, 'ag_text_classifier')

In [42]:
predictor.predict(sentence="Canadian Press - VANCOUVER (CP) - The sister of a man who died after a violent confrontation with police has demanded the city's chief constable resign for defending the officer involved.")

{'class_probabilities': [2.405789976955841e-11,
  4.568966868257265e-15,
  1.0,
  3.932658886041107e-12],
 'label': '1',
 'logits': [2.251049280166626,
  -6.3178839683532715,
  26.70160675048828,
  0.43990111351013184]}

# Interpret

In [21]:
@Predictor.register('ag_text_classifier_with_input_red')
class InputReductionTextClassifierPredictor(AGNewsClassifier):
    
    def predict_json(self, json_dict: JsonDict) -> JsonDict:
        predictor = AGNewsClassifier(self._model, self._dataset_reader)
        prediction = predictor.predict(sentence=json_dict['Description'])

        attacker = InputReduction(predictor)
        attack = attacker.attack_from_json(inputs=json_dict,
                                           input_field_to_attack='tokens',
                                           grad_input_field='grad_input_1',
                                           ignore_tokens=None)

        return {'prediction': prediction, 'input_reduction_output': attack}

In [22]:
predictor_with_input_red = Predictor.from_archive(archive, 'ag_text_classifier_with_input_red')

In [23]:
input = "The Cleveland Indians pulled within one game of the AL Central lead by beating the Minnesota Twins, 7-1, Saturday night with home runs by Travis Hafner and Victor Martinez."

In [24]:
predictor_with_input_red.predict(input)

{'input_reduction_output': {'final': [['pulled', 'Hafner']],
  'original': ['The',
   'Cleveland',
   'Indians',
   'pulled',
   'within',
   'one',
   'game',
   'of',
   'the',
   'AL',
   'Central',
   'lead',
   'by',
   'beating',
   'the',
   'Minnesota',
   'Twins',
   ',',
   '7',
   '-',
   '1',
   ',',
   'Saturday',
   'night',
   'with',
   'home',
   'runs',
   'by',
   'Travis',
   'Hafner',
   'and',
   'Victor',
   'Martinez',
   '.']},
 'prediction': {'class_probabilities': [0.000256297062151134,
   0.9993062019348145,
   0.0002524169394746423,
   0.00018510186055209488],
  'label': '2',
  'logits': [-1.1653821468353271,
   7.103096961975098,
   -1.1806373596191406,
   -1.4908137321472168]}}

In [25]:
result = predictor_with_input_red.predict(input)
tokens = result['input_reduction_output']['original']

grad = SimpleGradient(predictor).saliency_interpret_from_json({'Description':input})
gradient_list = grad['instance_1']['grad_input_1']

# Visualisation

Source: https://adataanalyst.com/machine-learning/highlight-text-using-weights/

In [26]:
import html
import random
from IPython.core.display import display, HTML

In [27]:
# Prevent special characters like & and < to cause the browser to display something other than what you intended.
def html_escape(text):
    return html.escape(text)

In [28]:
max_alpha = 0.4 
highlighted_text = []
for i in range(len(tokens)):
    weight = gradient_list[i]
    highlighted_text.append('<span style="background-color:rgba(135,206,250,' + str(weight / max_alpha) + ');">' + html_escape(tokens[i]) + '</span>')
highlighted_text = ' '.join(highlighted_text)

In [29]:
print(display(HTML(highlighted_text)))

None


# Checklist

In [32]:
sample = []
for row in test_set.head(10).itertuples():
  sample.append(row[3])

In [43]:
for sentence in sample:
  result = predictor_with_input_red.predict(sentence)
  tokens = result['input_reduction_output']['original']

  grad = SimpleGradient(predictor).saliency_interpret_from_json({'Description':sentence})
  gradient_list = grad['instance_1']['grad_input_1']
  max_alpha = 0.4 
  highlighted_text = []
  for i in range(len(tokens)):
      weight = gradient_list[i]
      highlighted_text.append('<span style="background-color:rgba(135,206,250,' + str(weight / max_alpha) + ');">' + html_escape(tokens[i]) + '</span>')
  highlighted_text = ' '.join(highlighted_text)
  print(display(HTML(highlighted_text)))

None


KeyboardInterrupt: ignored

In [35]:
pdata = list(nlp.pipe(sample))

In [45]:
for i in range(10):
  sentence = sample[i]
  if Perturb.contractions(sample[i]) != []:
  
    result = predictor_with_input_red.predict(sentence)
    tokens = result['input_reduction_output']['original']

    grad_orig = SimpleGradient(predictor).saliency_interpret_from_json({'Description':sentence})
    gradient_list_orig = np.array(grad_orig['instance_1']['grad_input_1'])

    max_alpha = 0.4 
    highlighted_text = []
    for i in range(len(tokens)):
        weight = gradient_list_orig[i]
        highlighted_text.append('<span style="background-color:rgba(135,206,250,' + str(weight / max_alpha) + ');">' + html_escape(tokens[i]) + '</span>')
    highlighted_text = ' '.join(highlighted_text)
    print(display(HTML(highlighted_text)))

    perturbed_sentence = Perturb.contractions(sentence)[0]

    result = predictor_with_input_red.predict(perturbed_sentence)
    tokens = result['input_reduction_output']['original']

    grad_pert = SimpleGradient(predictor).saliency_interpret_from_json({'Description': perturbed_sentence})
    gradient_list_pert = np.array(grad_pert['instance_1']['grad_input_1'])

    max_alpha = 0.4 
    highlighted_text = []
    for i in range(len(tokens)):
        weight = gradient_list_pert[i]
        highlighted_text.append('<span style="background-color:rgba(135,206,250,' + str(weight / max_alpha) + ');">' + html_escape(tokens[i]) + '</span>')
    highlighted_text = ' '.join(highlighted_text)
    print(display(HTML(highlighted_text)))

None


None


None


None


KeyboardInterrupt: ignored